# **Análise e qualidade de dados com Pyhton**

https://towardsdatascience.com/a-data-quality-test-approach-with-python-to-identify-non-standard-character-patterns-864affcb9195

https://www.datapine.com/blog/data-quality-management-and-metrics/

https://www.sas.com/pt_br/insights/analytics/processamento-de-linguagem-natural.html

https://blog.cedrotech.com/linguagem-natural-entenda-o-que-e-e-a-importancia-na-era-de-chatbots#:~:text=Considerando%20que%20um%20computador%20seja,frases%20%C3%A9%20algo%20bastante%20complexo.

# Análise Exploratória

O primiero passo deste projeto é a análise exploratória dos dados, que facilita o entendimento dos dados, investiga e resume suas principais características.

In [55]:
# Biblioteca:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn 
import nltk
import re, string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
nltk.download('all')

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger is already up-
[nltk_data]    |       to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger_ru is already
[nltk_data]    |       up-to-date!
[nltk_data]    | Downloading package basque_grammars to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package basque_grammars is already up-to-date!
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Pac

True

In [56]:
# Dados:
df = pd.read_excel('/content/Amostra dos dados.xlsx')

# Mostra as 5 primeiras linhas:
df.head()

DECISÃO
0  DECISÃO<br>Trata-se de habeas corpus impetrado...
1  DECISÃO<br>Cuida-se de recurso especial interp...
2  DECISÃO<br>Cuida-se de recurso ordinário em ma...
3  DECISÃO<br>Trata-se de habeas corpus impetrado...
4  DECISÃO<br>Cuida-se de recurso especial interp...

In [57]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 1 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   DECISÃO  17 non-null     object
dtypes: object(1)
memory usage: 288.0+ bytes


In [58]:
df.isnull().sum()

DECISÃO    3
dtype: int64

In [59]:
df.value_counts('DECISÃO')

DECISÃO
DECISÃO<br>1. Trata-se de agravo interposto por CAIXA DE PREVIDENCIA DOS FUNCS DO BANCO DO BRASIL, contra decisão que não admitiu o seu recurso especial, por sua vez manejado com fulcro no art. 105, III, "a", da Constituição Federal, em face de acórdão do Tribunal de Justiça do Distrito Federal e Territórios, assim ementado (fls. 681-682):<br>APELAÇÕES CÍVEIS. DIREITO PREVIDENCIÁRIO E DIREITO CIVIL. AÇÃO REVISIONAL DE BENEFÍCIO PREVIDENCIÁRIO. INCLUSÃO DE HORAS EXTRAS NO CÁLCULO DO BENEFÍCIO. PARCELAS RECONHECIDAS NA JUSTIÇA DO TRABALHO. RESP 1.312.736/RS.<br>RECURSO REPETITIVO. MODULAÇÃO DOS EFEITOS. LEGITIMIDADE PASSIVA DO BANCO DO BRASIL E LEGITIMIDADE ATIVA DO PARTICIPANTE. COMPETÊNCIA DA JUSTIÇA COMUM. COISA JULGADA NA RECLAMAÇÃO TRABALHISTA. AÇÕES COM OBJETOS DISTINTOS. PRESCRIÇÃO QUINQUENAL. TEORIA DA ACTIO NATA.<br>RECOMPOSIÇÃO DA RESERVA MATEMÁTICA. NECESSIDADE DE ESTUDO TÉCNICO ATUARIAL. POSSIBILIDADE DE COMPENSAÇÃO DE VALORES ENTRE PARTICIPANTE E ENTIDADE. OBSERVÂNCI

Problemas identificados: para serem processados os dados precisam estar limpos e passar pelo PLN

# Limpeza dos dados

1.   Remover linhas com valores ausentes;


In [60]:
df.loc[(df['DECISÃO'].isnull())]

DECISÃO
10     NaN
15     NaN
17     NaN

In [61]:
df=df.dropna(subset=['DECISÃO'])

df.reset_index(inplace=True, drop=True)

df.shape

(17, 1)

2.   Remover DECISÃO<br


In [62]:
df['DECISÃO'] = df['DECISÃO'].astype(str)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [63]:
def remover(decisao):
    decisao = decisao.replace('DECISÃO<br>', ' ')
    return decisao

df['Limpo'] = df['DECISÃO'].apply(remover)
df.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


DECISÃO  \
0  DECISÃO<br>Trata-se de habeas corpus impetrado...   
1  DECISÃO<br>Cuida-se de recurso especial interp...   
2  DECISÃO<br>Cuida-se de recurso ordinário em ma...   
3  DECISÃO<br>Trata-se de habeas corpus impetrado...   
4  DECISÃO<br>Cuida-se de recurso especial interp...   

                                               Limpo  
0   Trata-se de habeas corpus impetrado em benefí...  
1   Cuida-se de recurso especial interposto por C...  
2   Cuida-se de recurso ordinário em mandado de s...  
3   Trata-se de habeas corpus impetrado em benefí...  
4   Cuida-se de recurso especial interposto por N...

3. Remover acentos

In [64]:
def acentos(unidade):
    unidade = unidade.replace('.', '').replace('ç', 'c').replace('ô', 'o').replace('é', 'e').replace('ã', 'a').replace('õ', 'o').replace('ó', 'o').replace('ê', 'e').replace('à', 'a').replace('í', 'i').replace('-', ' ').replace('á', 'a').replace('ù', 'u').replace('ú', 'u')
    return unidade

df['Limpo'].apply(acentos)
df.head()

DECISÃO  \
0  DECISÃO<br>Trata-se de habeas corpus impetrado...   
1  DECISÃO<br>Cuida-se de recurso especial interp...   
2  DECISÃO<br>Cuida-se de recurso ordinário em ma...   
3  DECISÃO<br>Trata-se de habeas corpus impetrado...   
4  DECISÃO<br>Cuida-se de recurso especial interp...   

                                               Limpo  
0   Trata-se de habeas corpus impetrado em benefí...  
1   Cuida-se de recurso especial interposto por C...  
2   Cuida-se de recurso ordinário em mandado de s...  
3   Trata-se de habeas corpus impetrado em benefí...  
4   Cuida-se de recurso especial interposto por N...

4. Converter para minúsculas, retirar remover pontuações, stopwords, LEMMATIZATION

In [65]:
def preprocess(text):
    text = text.lower() 
    text = text.strip()  
    text = re.compile('<.*?>').sub('', text) 
    text = re.compile('[%s]' % re.escape(string.punctuation)).sub(' ', text)  
    text = re.sub('\s+', ' ', text)  
    text = re.sub(r'\[[0-9]*\]',' ',text) 
    text=re.sub(r'[^\w\s]', '', str(text).lower().strip())
    text = re.sub(r'\d',' ',text) 
    text = re.sub(r'\s+',' ',text)
    return text


# STOPWORD
def stopword(string):
    a= [i for i in string.split() if i not in nltk.corpus.stopwords.words('portuguese')]
    return ' '.join(a)

#LEMMATIZATION
wl = WordNetLemmatizer()
 

def get_wordnet_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN

def lemmatizer(string):
    word_pos_tags = nltk.pos_tag(word_tokenize(string))
    a=[wl.lemmatize(tag[0], get_wordnet_pos(tag[1])) for idx, tag in enumerate(word_pos_tags)] 
    return " ".join(a)

In [66]:
def finalpreprocess(string):
    return lemmatizer(stopword(preprocess(string)))
df['Limpo'].apply(lambda x: finalpreprocess(x))

0     trata habeas corpus impetrado benefício valter...
1     cuida recurso especial interposto ccx empreend...
2     cuida recurso ordinário mandado segurançainter...
3     trata habeas corpus impetrado benefício marcel...
4     cuida recurso especial interposto nicolau barr...
5     cuida recurso especial interposto nicolau barr...
6     cuida recurso especial interposto juliano foia...
7     trata recurso especial interposto sérgio antôn...
8     trata recurso especial interposto banco brasil...
9     trata agravo interposto caixa previdencia func...
10    trata recurso especial interposto telma lucia ...
11    trata recurso especial interposto elia albino ...
12    trata habeas corpus impetrado benefício caio h...
13    trata agravo interposto estado rio janeiro con...
14    trata agravos manejados caixa econômica federa...
15    trata recurso extraordinário interposto otavio...
16    trata agravo interposto ilka terezinha migott ...
Name: Limpo, dtype: object

# Resultado

In [67]:
df.head()

DECISÃO  \
0  DECISÃO<br>Trata-se de habeas corpus impetrado...   
1  DECISÃO<br>Cuida-se de recurso especial interp...   
2  DECISÃO<br>Cuida-se de recurso ordinário em ma...   
3  DECISÃO<br>Trata-se de habeas corpus impetrado...   
4  DECISÃO<br>Cuida-se de recurso especial interp...   

                                               Limpo  
0   Trata-se de habeas corpus impetrado em benefí...  
1   Cuida-se de recurso especial interposto por C...  
2   Cuida-se de recurso ordinário em mandado de s...  
3   Trata-se de habeas corpus impetrado em benefí...  
4   Cuida-se de recurso especial interposto por N...

In [68]:
# Salvando:

df.to_excel("STJ_Limpo.xlsx")  